In [1]:
import re
import time
import json
import uuid
import redis
import datetime
import pandas as pd

from sqlalchemy import create_engine
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

In [2]:
pool = redis.ConnectionPool(host='172.17.0.2', port=6379, decode_responses=True, db=1)
redis_con = redis.Redis(connection_pool=pool)

addr = '172.17.0.3'
user = 'root'
passwd = '123456'

mysql_engin = 'mysql+pymysql://'+user+':'+passwd+'@'+addr+'/hot?charset=utf8'
engine = create_engine(mysql_engin)

In [48]:
import requests

ori = 'https://api.onebound.cn/taobao/api_call.php?key=tel18349125880&secret=20201009&api_name=item_search&q=小黑裙&page_size=100&page=1&sort=_sale'
# data = requests.get(src).content

In [62]:
import json

theme = pd.read_csv('theme.csv')
theme = theme.loc[(theme['platform'] == 'taobao'),:].reset_index(drop=True)
theme.son_theme.values
final = pd.DataFrame()
for son_theme in theme.son_theme.values:
    src = ori.replace(re.findall('&q=(.*?)&',ori)[0],son_theme)
    for i in range(1,11):
        print(src)
        time.sleep(5)
        data = requests.get(src).content
        data = pd.DataFrame(json.loads(data)['items']['item'])
        data['id'] = pd.Series([str(uuid.uuid4()) for _ in range(0, len(data))])
        final = final.append(data).reset_index(drop=True)
        src = src.replace(re.findall('&page=(.*?)&',src)[0],str(i+1))
    final.to_sql('links', con=engine,  if_exists='append', index=False)

https://api.onebound.cn/taobao/api_call.php?key=tel18349125880&secret=20201009&api_name=item_search&q=毛衣外套&page_size=100&page=1&sort=_sale
https://api.onebound.cn/taobao/api_call.php?key=tel28349225880&secret=20202009&api_name=item_search&q=毛衣外套&page_size=200&page=2&sort=_sale


KeyError: 'items'

In [63]:
json.loads(data)

{'error': '你不是正版用户无权使用API接口！您的服务器IP已被记录。请联系QQ:583964941购买正版授权。',
 'reason': '你不是正版用户无权使用API接口！您的服务器IP已被记录。请联系QQ:583964941购买正版授权。',
 'error_code': '4003',
 'success': 0,
 'cache': 0,
 'api_info': 'today:0 max:',
 'execution_time': 0.05,
 'server_time': 'Beijing/2020-10-09 22:02:17',
 'call_args': [],
 'api_type': 'taobao',
 'server_memory': '4.83MB',
 'request_id': '15f806d69d2162'}

In [43]:
temp.to_sql('links', con=engine,  if_exists='append', index=False)